# importing necessary libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import re
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import json 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer

# loading meta data and converting into data frame

In [2]:

data=[]

file = "meta_All_Beauty.jsonl"
with open(file, 'r') as fp:
    for line in fp:
        data.append(json.loads(line.strip()))

# Convert the list of dictionaries to a DataFrame
meta = pd.DataFrame(data)


In [3]:
meta.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,[],{'Package Dimensions': '7.1 x 5.5 x 3 inches; ...,B01CUPMQZE,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Yes To,[],"{'Item Form': 'Powder', 'Skin Type': 'Acne Pro...",B076WQZGPM,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Levine Health Products,[],{'Manufacturer': 'Levine Health Products'},B000B658RI,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Cherioll,[],"{'Brand': 'Cherioll', 'Item Form': 'Powder', '...",B088FKY3VD,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Precision,[],{'UPC': '644287689178'},B07NGFDN6G,None


In [4]:
meta.rename(columns={'title': 'product_description'}, inplace=True)

In [5]:
#selecting relevant features from meta data
meta=meta[['product_description','average_rating','store', 'parent_asin','rating_number']]

# loading reviews data and converting into dataframe

In [6]:

data=[]

file = "All_Beauty.jsonl"
with open(file, 'r') as fp:
    for line in fp:
        data.append(json.loads(line.strip()))

# Convert the list of dictionaries to a DataFrame
review_df = pd.DataFrame(data)


# print top 5 rows of data

In [7]:
review_df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True


In [8]:
#selecting relevant features from review data
review_df=review_df[['user_id','text','title','rating','verified_purchase','helpful_vote','parent_asin','timestamp']]

# combining both datasets based on parent_asin

In [9]:
merged_df = review_df.merge(meta, on='parent_asin', how='inner')

In [10]:
merged_df.head()

,user_id,text,title,rating,verified_purchase,helpful_vote,parent_asin,timestamp,product_description,average_rating,store,rating_number
0,AGKHLEW2SOWHNMFQIJGBECAF7INQ,This spray is really nice. It smells really go...,Such a lovely scent but not overpowering.,5.0,True,0,B00YQ6X8EO,1588687728923,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384
1,AGF2RF5HBCXKEOLTF2LXZTVFYFVQ,This does not work as well as other sea mist h...,Not worth it.,1.0,True,0,B00YQ6X8EO,1667089254927,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384
2,AGPWM36OKQCL3PM4MXZNLP4VVJKA,Makes my hair smell amazing and I love the tex...,love,5.0,True,1,B00YQ6X8EO,1455021933000,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384
3,AGVVMZD2DJQ7Z3KJJXHFKQHSX7HA,Pros: smells AMAZING like a beachy beverage! (...,Mermaid hair in a bottle,5.0,True,0,B00YQ6X8EO,1623855338155,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384
4,AE5VLXANR3ZCUZCQAOBJTIEGMZGA,This product smells divine but it weighs down ...,It makes my curly/wavy hair way too smooth and...,2.0,True,0,B00YQ6X8EO,1623342316264,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384


In [11]:
merged_df.shape

(701528, 12)

# selecting products that contains  more than  2 reviews 

In [12]:
# Get the value counts of the 'parent_asin' column
store_counts = merged_df['parent_asin'].value_counts()

# Filter the 'store' values where the count is greater than 100
valid_stores = store_counts[store_counts > 2].index

# Filter the dataset based on the valid 'store' values
merged_df = merged_df[merged_df['parent_asin'].isin(valid_stores)]

# text cleaning

In [13]:
# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join tokens back to a cleaned string
    return ' '.join(tokens)

# Apply the cleaning function to the 'text' column
merged_df['cleaned_text'] = merged_df['text'].apply(clean_text)

# Display a sample of the cleaned text
print(merged_df[['text', 'cleaned_text']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ghufr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ghufr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ghufr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                text  \
0  This spray is really nice. It smells really go...   
1  This does not work as well as other sea mist h...   
2  Makes my hair smell amazing and I love the tex...   
3  Pros: smells AMAZING like a beachy beverage! (...   
4  This product smells divine but it weighs down ...   

                                        cleaned_text  
0  spray really nice smell really good go really ...  
1  work well sea mist hair product give beach hai...  
2           make hair smell amazing love texture add  
3  pro smell amazing like beachy beverage id say ...  
4  product smell divine weighs hair want curlwave...  


# sentiment analysis using vader

In [14]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Ensure required NLTK data is downloaded
nltk.download('vader_lexicon')

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to calculate sentiment score using VADER
def calculate_sentiment_score(text):
    sentiment = analyzer.polarity_scores(text)  # Get sentiment scores
    return sentiment['compound']  # Return the compound score as overall sentiment

# Apply the function to calculate sentiment scores for each review
merged_df['sentiment_score'] = merged_df['cleaned_text'].apply(calculate_sentiment_score)






[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ghufr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
merged_df['sentiment_score'].describe()

count    614214.000000
mean          0.487533
std           0.435364
min          -0.994900
25%           0.226300
50%           0.624900
75%           0.840200
max           0.999700
Name: sentiment_score, dtype: float64

In [16]:
# Define Dynamic Sentiment Thresholds
# Here we define "extreme" based on quantiles of the sentiment scores
lower_threshold = merged_df['sentiment_score'].quantile(0.25)  # 25th percentile for negativity
upper_threshold = merged_df['sentiment_score'].quantile(0.75)  # 75th percentile for positivity


# Feature 2: Uppercase Word Count
merged_df['uppercase_count'] = merged_df['text'].apply(lambda x: sum(1 for word in x.split() if word.isupper()))

# Feature 3: Review Length
merged_df['review_length'] = merged_df['text'].apply(lambda x: len(x.split()))


 # creating target feature (review wise)

In [17]:

# Apply conditions to classify reviews
merged_df['extremist'] = (
    (merged_df['rating'].isin([1, 4, 5])) &
    (merged_df['verified_purchase'] == True) &
    ((merged_df['sentiment_score'] < lower_threshold) | (merged_df['sentiment_score'] > upper_threshold))
).map({True: 'extremist', False: 'moderate'})





In [18]:
# Analyze the extremist reviews count
extremist_count = merged_df['extremist'].value_counts()
print(f"Extremist Reviews Count:\n{extremist_count}")

Extremist Reviews Count:
extremist
moderate     388304
extremist    225910
Name: count, dtype: int64


# Group by 'product' and perform majority voting on the 'extremist' column

In [20]:


# Step 1: Group by 'product' and perform majority voting on the 'extremist' column
majority_extremist = (
    merged_df.groupby('parent_asin')['extremist']
    .apply(lambda x: x.value_counts().idxmax())
    .reset_index()
    .rename(columns={'extremist': 'majority_extremist'})
)

# Step 2: Merge the majority result back to the original dataframe
merged_df = merged_df.merge(majority_extremist, on='parent_asin')

# Display the updated dataframe
merged_df.head()


,user_id,text,title,rating,verified_purchase,helpful_vote,parent_asin,timestamp,product_description,average_rating,store,rating_number,cleaned_text,sentiment_score,uppercase_count,review_length,extremist,majority_extremist
0,AGKHLEW2SOWHNMFQIJGBECAF7INQ,This spray is really nice. It smells really go...,Such a lovely scent but not overpowering.,5.0,True,0,B00YQ6X8EO,1588687728923,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384,spray really nice smell really good go really ...,0.7846,4,61,moderate,moderate
1,AGF2RF5HBCXKEOLTF2LXZTVFYFVQ,This does not work as well as other sea mist h...,Not worth it.,1.0,True,0,B00YQ6X8EO,1667089254927,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384,work well sea mist hair product give beach hai...,0.2732,0,21,moderate,moderate
2,AGPWM36OKQCL3PM4MXZNLP4VVJKA,Makes my hair smell amazing and I love the tex...,love,5.0,True,1,B00YQ6X8EO,1455021933000,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384,make hair smell amazing love texture add,0.8402,1,12,moderate,moderate
3,AGVVMZD2DJQ7Z3KJJXHFKQHSX7HA,Pros: smells AMAZING like a beachy beverage! (...,Mermaid hair in a bottle,5.0,True,0,B00YQ6X8EO,1623855338155,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384,pro smell amazing like beachy beverage id say ...,0.9397,2,83,extremist,moderate
4,AE5VLXANR3ZCUZCQAOBJTIEGMZGA,This product smells divine but it weighs down ...,It makes my curly/wavy hair way too smooth and...,2.0,True,0,B00YQ6X8EO,1623342316264,Herbivore - Natural Sea Mist Texturizing Salt ...,4.3,HERBIVORE,384,product smell divine weighs hair want curlwave...,0.8750,2,57,moderate,moderate


# feature extraction:

# review count for each product

In [21]:
# Group by product asin to count the number of reviews per product 
review_count = merged_df.groupby(['parent_asin']).size().reset_index(name='review_count')

# Merge the review_count with the original dataset to add the review count as a new column
merged_df = merged_df.merge(review_count, on=['parent_asin'], how='left')


# group_time_window

In [22]:
# Convert timestamp to datetime for better interpretation
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'], unit='ms')

# Group by 'product' and calculate the Group Time Window (GT)
group_time_window = merged_df.groupby('parent_asin')['timestamp'].agg(['min', 'max'])
group_time_window['Group_Time'] = (group_time_window['max'] - group_time_window['min']).dt.days

# Display the results
group_time_window = group_time_window.reset_index()
group_time_window = group_time_window[['parent_asin', 'Group_Time']].sort_values(by='Group_Time', ascending=True)


# Merge the GT values back into the original dataset
merged_df = merged_df.merge(group_time_window, on='parent_asin', how='left')

print(group_time_window.head())

      parent_asin  Group_Time
44160  B09YLXRSNN           0
44043  B09WKFTWLG           0
40824  B08ZSGJ1H1           0
33165  B0881XHZ99           0
33123  B087XBFPRQ           0


# avg uppercase_count for each product

In [23]:
# Calculate Average Upvotes (avg_upvotes) for each product
average_uppercase_count = (
    merged_df.groupby(['parent_asin'])['uppercase_count']
    .mean()
    .reset_index(name='average_uppercase_count')
)

# Merge the average upvotes back into the original dataset
merged_df = merged_df.merge(average_uppercase_count, on=[ 'parent_asin'], how='left')




# avg review_length for each product

In [24]:
# Calculate Average Upvotes (avg_upvotes) for each product
average_review_length = (
    merged_df.groupby(['parent_asin'])['review_length']
    .mean()
    .reset_index(name='average_review_length')
)

# Merge the average upvotes back into the original dataset
merged_df = merged_df.merge(average_review_length, on=[ 'parent_asin'], how='left')




# avg_upvotes

In [25]:
# Calculate Average Upvotes (avg_upvotes) for each product
average_upvotes = (
    merged_df.groupby(['parent_asin'])['helpful_vote']
    .mean()
    .reset_index(name='avg_upvotes')
)

# Merge the average upvotes back into the original dataset
merged_df = merged_df.merge(average_upvotes, on=[ 'parent_asin'], how='left')




# Average Sentiment

In [26]:
# Compute Average Sentiment for each product
average_sentiment = (
    merged_df.groupby([ 'parent_asin'])['sentiment_score']
    .mean()
    .reset_index(name='avg_sentiment')
)
# Merge the average upvotes back into the original dataset
merged_df = merged_df.merge(average_sentiment, on=[ 'parent_asin'], how='left')


# verified purchase ratio

In [27]:
# Calculate verified purchase ratio for each parent_asin
verified_purchase_ratio = (
    merged_df.groupby('parent_asin')['verified_purchase']
    .agg(['sum', 'count'])  # Sum of True values and total count
    .rename(columns={'sum': 'verified_true_count', 'count': 'total_reviews'})
)

# Add a ratio column
verified_purchase_ratio['verified_purchase_ratio'] = (
    verified_purchase_ratio['verified_true_count'] / verified_purchase_ratio['total_reviews']
)



# Display the result
verified_purchase_ratio.reset_index().head()


,parent_asin,verified_true_count,total_reviews,verified_purchase_ratio
0,0124784577,3,3,1.000000
1,069267599X,40,41,0.975610
2,0816091846,12,14,0.857143
3,0977949125,2,3,0.666667
4,0985393300,9,30,0.300000


In [28]:
# Merge the average upvotes back into the original dataset
merged_df = merged_df.merge(verified_purchase_ratio, on=[ 'parent_asin'], how='left')


# rating deviation for each product

In [29]:
# Step 1: Calculate the mean rating for each product
product_avg_rating = merged_df.groupby('parent_asin')['rating'].mean().reset_index()
product_avg_rating.rename(columns={'rating': 'mean_rating'}, inplace=True)

# Step 2: Merge the mean rating back into the original DataFrame
merged_df = merged_df.merge(product_avg_rating, on='parent_asin')

# Step 3: Compute absolute deviation from the mean
merged_df['absolute_deviation'] = abs(merged_df['rating'] - merged_df['mean_rating'])

# Step 4: Group by product and calculate the average deviation
product_rating_deviation = merged_df.groupby('parent_asin')['absolute_deviation'].mean().reset_index()
product_rating_deviation.rename(columns={'absolute_deviation': 'rating_deviation'}, inplace=True)

# Display the resulting product-wise rating deviation
product_rating_deviation.head()


,parent_asin,rating_deviation
0,0124784577,0.888889
1,069267599X,0.385485
2,0816091846,0.612245
3,0977949125,0.000000
4,0985393300,0.180000


In [30]:
merged_df = merged_df.merge(product_rating_deviation, on=[ 'parent_asin'], how='left')

# selecting final features

In [31]:
merged_df=merged_df[['parent_asin', 'review_count','average_uppercase_count','average_review_length',
       'Group_Time', 'avg_upvotes', 'avg_sentiment', 'verified_true_count',
       'total_reviews', 'verified_purchase_ratio', 'mean_rating',
        'rating_deviation','majority_extremist']]

In [32]:
final=merged_df.drop_duplicates()

In [33]:
final['majority_extremist'].value_counts()

majority_extremist
moderate     33645
extremist    11480
Name: count, dtype: int64

In [34]:
final=final.reset_index(drop=True)

In [35]:
final.to_csv('preprocessed_labeled_data.csv')